Az osztály(oka)t mentsd a __src/linear_regression__ modul __LinearRegressions.py__ fájljába
Használható modulok: _pathlib, pandas, typing, str, numpy, scipy.stats t_ és _norm_, valamint a _scipy.optimize_ _minimize_ osztályai 

In [ ]:
%load_ext nb_black
%load_ext autoreload
%autoreload 2

In [ ]:
from pathlib import Path

In [ ]:
datalib = Path.cwd().parent.joinpath('data')

### Adatelőkészítés (0 pont)
1., Olvasd be a data mappa __sp500.parquet__ nevű fájlját egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
2., Olvasd be az __ff_factors.parquet__ fájlt egy DataFrame-be. A betöltéshez használt engine paramétere legyen <u>fastparquet</u>
3., Kapcsold össze a két DataFrame-t egy új DataFrame-be. Az összekapcsolás módja, hogy a hozam adatokra balról kapcsoljuk rá a factor adatokat a __'Date' elsődleges kulcs__ alapján.
4., Készíts egy új __'Excess Return'__ nevű oszlopot, ami a havi hozamok és a kockázat mentes hozam (RF) különbsége
5., <u>Rendezd sorba dátum szerint az adatokat</u>, majd generálj egy új oszlopot (__'ex_ret_1'__), amely minden ticker ('Symbol') esetén 1-el eltolja az Excess Return értékeit olyan módon, hogy minden sorban szerepeljen a <u>következő időszaki Excess Return</u> érték. 
6., A meglévő adathalmazt írd felül olyan módon, hogy egyszer <u>törlöd az össze olyan sort</u>, amely az __'ex_ret_1' oszlopban hiányos__, majd ezt követően, törlöd az összes olyan sort, ami a __'HML' oszlopban hiányos__.


3. feladat segítség
![Joined data](../resources/weekly6/joined_data.jpg)

5. feladat segítség
![new column](../resources/weekly6/ex_ret_1.jpg)

### Modell összeállítás (7 pont)

A meglévő adatokból válaszd ki a __Amazon részvényhez tartozó sorokat (AMZN)__ és töröld a tickereket tartalmazó oszlopot. <u>Amennyiben működött, önellenőrzésre használhatod a legutóbbi zárthelyin írt osztályt is.</u>

7., Készíts egy új __LinearRegressionML__ elnevezésű osztályt. Definiáld benne a __\_\_init\_\___ nevű függvényt, amely bemenetként 2 DataFrame-t kap amelyeket ments le a left_hand_side és right_hand_side elnevezésű változókba. Az egyik DataFrame fogja tartalmazni a következő hónap többlet hozamait (__left_hand_side__), a másik a piaci hozamokat (Mkt-RF), az SMB és a HML értékeket (__right_hand_side__).

8., Egésztsd ki az osztályt egy __fit__ metódussal, ami ML elvű becslést hajt végre. Figyelj oda, hogy a regresszió futtatása során konstans (alfa / béta_0) is szerepeljen a predictor változók között, amely az első változó legyen. __A feladatot numerikus optimalizálással old meg, analítikus megoldásra nem ját pont__

9., Egészítsd ki az osztályt egy __get_params__ metódussal, ami visszaadja a becsült modell béta paramétereinek értékeit. A visszakapott pandas Series típusú adatban az oszlop neve legyen __Beta coefficients__. 

10., Egészítsd ki az osztályt egy __get_pvalues__ metódussal, ami visszaadja a becsült modell paraméterekhez tartozó p értékeket. A visszakapott pandas Series típusú adatban az oszlop neve legyen: __P-values for the corresponding coefficients__. A p értéket t-statisztika alapján számold ki. A p-érték kiszámításánál figyelj, hogy  alkalmazd a <u>min(value, 1-value) * 2</u> képletet.

11., Egészítse ki az osztályt egy __get_model_goodness_values__ metódussal, ami visszadja a centrált és a módosított R-négyzet értékeket. A visszatérési típus string legyen, a visszaadandó szöveg: __Centered R-squared: crs, Adjusted R-squared: ars__, ahol crs és ars helyére 3 tizedesjegyre kerekítve (__:.3f__) add meg a hozzájuk tartozó értékeket. <u>Ha a regresszorok számába eredetileg beleszámítottad a konstanst is, akkor a módosított R-négyzet számítás nevezőjében nincs szükség a __-1__-es tagra</u>.

#### Kiegészítés:
- Mivel a célparaméterek kicsik, a minimalizálás kiinduló értékeit érdemes egységesen, minden paraméter számára __0.1__-re állítani.
- Javasolt a __L-BFGS-B__ optimalizáló alkalmazása
- Ne felejtsd el, hogy az MLE __variancia becslése torzított__. A kapott eredményeket ennek megfelelően korrigáld.

In [ ]:
    def get_pvalues(self):
        # Calculate p-values for coefficients
        self.fit()
        degrees_of_freedom = len(self.observed_values) - self.design_matrix.shape[1]
        residuals = self.observed_values - self.design_matrix @ self.coefficients
        residual_var = (residuals @ residuals) / degrees_of_freedom
        t_stat = self.coefficients / np.sqrt(np.diag(residual_var * np.linalg.inv(self.design_matrix.T @ self.weights_inv @ self.design_matrix)))

        # Calculate two-tailed p-values using the t distribution
        p_values = pd.Series([min(value, 1 - value) * 2 for value in t.cdf(-np.abs(t_stat), df=degrees_of_freedom)],
                             name='P-values for the corresponding coefficients')
        return p_values

In [ ]:
    def get_pvalues(self):
        self.fit()
        degrees_of_freedom = len(self.left_hand_side) - len(self.coefficients)
        residuals = self.left_hand_side.values.flatten() - np.dot(np.column_stack((np.ones_like(self.left_hand_side), self.right_hand_side.values)), self.coefficients)
        residual_var = (residuals @ residuals) / degrees_of_freedom
        t_stat = self.coefficients / np.sqrt(np.diag(residual_var * np.linalg.inv(np.dot(np.column_stack((np.ones_like(self.left_hand_side), self.right_hand_side.values)).T, np.column_stack((np.ones_like(self.left_hand_side), self.right_hand_side.values))))))

        # Calculate two-tailed p-values using the t distribution
        p_values = pd.Series([min(value, 1 - value) * 2 for value in norm.cdf(-np.abs(t_stat))],
                             name='P-values for the corresponding coefficients')
        return p_values




from typing import Optional
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.optimize import minimize
from scipy.stats import t

class LinearRegressionML:
    def __init__(self, left_hand_side: pd.DataFrame, right_hand_side: pd.DataFrame):
        self.left_hand_side = left_hand_side
        self.right_hand_side = right_hand_side
        self.coefficients = None

    def _negative_log_likelihood(self, params: np.ndarray) -> float:
        y = self.left_hand_side.values.flatten()
        X = np.column_stack((np.ones_like(y), self.right_hand_side.values))
        mu = np.dot(X, params)
        neg_log_likelihood = -np.sum(norm.logpdf(y, loc=mu))
        return neg_log_likelihood

    def fit(self) -> None:
        initial_guess = 0.1 * np.ones(self.right_hand_side.shape[1] + 1)
        result = minimize(self._negative_log_likelihood, initial_guess, method='L-BFGS-B')
        self.coefficients = result.x

    def get_params(self, include_intercept: Optional[bool] = True) -> pd.Series:
        if self.coefficients is None:
            raise ValueError("A modell még nincs illesztve. Kérlek, hajtsd végre a fit metódust.")

        param_names = ['Intercept' if include_intercept else '']
        param_names += [f'Beta_{i}' for i in range(1, len(self.coefficients))]

        return pd.Series(self.coefficients, index=param_names)



    def get_pvalues(self):
        self.fit()

        # Calculate residuals and degrees of freedom
        residuals = self.left_hand_side.values.flatten() - np.dot(np.column_stack((np.ones_like(self.left_hand_side), self.right_hand_side.values)), self.coefficients)
        degrees_of_freedom = len(self.left_hand_side) - len(self.coefficients)

        # Calculate MLE-based variance
        residual_var = np.sum((residuals / np.sqrt(degrees_of_freedom))**2) / len(residuals)

        # Calculate standard errors
        se = np.sqrt(np.diag(residual_var * np.linalg.inv(np.dot(np.column_stack((np.ones_like(self.left_hand_side), self.right_hand_side.values)).T, np.column_stack((np.ones_like(self.left_hand_side), self.right_hand_side.values))))))

        # Calculate t-statistic
        t_stat = self.coefficients / se

        # Calculate two-tailed p-values using the t-distribution
        p_values = 2 * (1 - t.cdf(np.abs(t_stat), degrees_of_freedom))

        # Create a DataFrame for p-values
        p_values_df = pd.DataFrame({'Coefficient': self.coefficient_names, 'P-value': p_values})

        return p_values_df